# Simple Impementation of E(n) Equivariant Graph Neural Networks

Original paper https://arxiv.org/pdf/2102.09844.pdf by Victor Garcia Satorras, Emiel Hoogeboom, Max Welling

In [ ]:
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# Load QM9 Dataset

In [ ]:
!git clone https://github.com/senya-ashukha/simple-equivariant-gnn.git
%cd simple-equivariant-gnn

C:\Users\sergk\Desktop\simple-equivariant-gnn


Cloning into 'simple-equivariant-gnn'...


In [ ]:
# QM9 is a dataset for Molecular Property Predictions http://quantum-machine.org/datasets/
# We will predict Highest occupied molecular orbital energy 
# https://en.wikipedia.org/wiki/HOMO_and_LUMO
# We use data loaders from the official repo

from qm9.data_utils import get_data, BatchGraph
train_loader, val_loader, test_loader, charge_scale = get_data(num_workers=0)

# Graph Representation

In [ ]:
batch = BatchGraph(iter(train_loader).next(), False, charge_scale)
batch

In the batch: num_graphs 96 num_nodes 1733
> .h 		 a tensor of nodes representations 		shape 1733 x 15
> .x 		 a tensor of nodes positions  			shape 1733 x 3
> .edges 	 a tensor of edges, a fully connected graph 	shape 30468 x 2
> .batch  	 a tensor of graph_ids for each node 		tensor([ 0,  0,  0,  ..., 95, 95, 95])

# Define Equivariant Graph Convs  & GNN

In [ ]:
def index_sum(agg_size, source, idx, cuda):
    """
        source is N x hid_dim [float]
        idx    is N           [int]
        
        Sums the rows source[.] with the same idx[.];
    """
    tmp = torch.zeros((agg_size, source.shape[1]))
    tmp = tmp.cuda() if cuda else tmp
    res = torch.index_add(tmp, 0, idx, source)
    return res

In [ ]:
class ConvEGNN(nn.Module):
    def __init__(self, in_dim, hid_dim, cuda=True):
        super().__init__()
        self.hid_dim = hid_dim
        self.cuda = cuda
        
        self.f_e = nn.Sequential(nn.Linear(in_dim * 2 + 1, hid_dim), 
                                 nn.SiLU(),
                                 nn.Linear(hid_dim, hid_dim), 
                                 nn.SiLU())
        
        self.f_inf = nn.Sequential(nn.Linear(hid_dim, 1), 
                                   nn.Sigmoid()) 
        
        self.f_h = nn.Sequential(nn.Linear(hid_dim + in_dim, hid_dim), 
                                 nn.SiLU(),
                                 nn.Linear(hid_dim, hid_dim))
    
    def forward(self, b):
        e_st, e_end = b.edges[:,0], b.edges[:,1]
        dists = torch.norm(b.x[e_st] - b.x[e_end], dim=1).reshape(-1, 1)
        
        tmp = torch.hstack([b.h[e_st], b.h[e_end], dists])
        m_ij = self.f_e(tmp)
        e_ij = self.f_inf(m_ij)
        
        m_i = index_sum(b.h.shape[0], e_ij * m_ij, b.edges[:,0], self.cuda)
        b.h = b.h + self.f_h(torch.hstack([b.h, m_i]))

        return b

In [ ]:
class NetEGNN(nn.Module):
    def __init__(self, in_dim=15, hid_dim=128, out_dim=1, n_layers=7, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim

        self.emb = nn.Linear(in_dim, hid_dim) 
        self.gnn = [ConvEGNN(hid_dim, hid_dim, cuda=cuda) for i in range(n_layers)]
        self.gnn = nn.Sequential(*self.gnn)
        
        self.pre_mlp = nn.Sequential(
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim))
        
        self.post_mlp = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, out_dim))

        if cuda: self.cuda()
        self.cuda = cuda
    
    def forward(self, b):
        b.h = self.emb(b.h)
        
        b = self.gnn(b)
        h_nodes = self.pre_mlp(b.h)
        h_graph = index_sum(b.nG, h_nodes, b.batch, self.cuda) 
        
        out = self.post_mlp(h_graph)
        return out

In [ ]:
epochs = 200
cuda = True

model = NetEGNN(n_layers=7, cuda=cuda)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, verbose=False)

# Training

In [ ]:
print('> start training')

tr_ys = train_loader.dataset.data['homo'] 
me, mad = torch.mean(tr_ys), torch.mean(torch.abs(tr_ys - torch.mean(tr_ys)))

if cuda:
    me = me.cuda()
    mad = mad.cuda()

train_loss = []
val_loss = []
test_loss = []

for epoch in range(epochs):
    print('> epoch %s:' % str(epoch).zfill(3), end=' ', flush=True) 
    start = time.time()

    batch_train_loss = []
    batch_val_loss = []
    batch_test_loss = []

    model.train()
    for batch in train_loader:
        batch = BatchGraph(batch, cuda, charge_scale)
        
        out = model(batch).reshape(-1)
        loss =  F.l1_loss(out,  (batch.y-me)/mad)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            loss =  F.l1_loss(out*mad+me, batch.y)

        batch_train_loss += [float(loss.data.cpu().numpy())]  
        
    train_loss += [np.mean(batch_train_loss)/0.001]
    
    print('train %.3f' % train_loss[-1], end=' ', flush=True)
    
    with torch.no_grad():
        model.eval()
        for batch in val_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_val_loss += [np.mean(loss)]
            
        val_loss += [np.mean(batch_val_loss)/0.001]
        
        print('val %.3f' % val_loss[-1], end=' ', flush=True)
        
        for batch in test_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_test_loss += [np.mean(loss)]

        test_loss += [np.mean(batch_test_loss)/0.001]
        
    end = time.time()

    print('test %.3f (%.1f sec)' % (test_loss[-1], end-start), flush=True)
    lr_scheduler.step()

> start training
> epoch 000: train 375.076 val 290.748 test 290.828 (209.0 sec)
> epoch 001: train 280.371 val 243.631 test 239.648 (207.6 sec)
> epoch 002: train 226.147 val 209.887 test 207.483 (208.5 sec)
> epoch 003: train 198.120 val 203.409 test 201.595 (208.5 sec)
> epoch 004: train 178.293 val 168.687 test 166.331 (208.8 sec)
> epoch 005: train 165.013 val 163.209 test 163.494 (209.3 sec)
> epoch 006: train 152.334 val 134.529 test 134.883 (209.5 sec)
> epoch 007: train 142.115 val 126.247 test 125.664 (209.0 sec)
> epoch 008: train 135.106 val 116.740 test 116.071 (209.1 sec)
> epoch 009: train 128.091 val 120.168 test 118.685 (209.5 sec)
> epoch 010: train 122.549 val 124.728 test 123.993 (209.2 sec)
> epoch 011: train 118.251 val 120.148 test 118.278 (209.1 sec)
> epoch 012: train 112.912 val 104.347 test 103.627 (209.5 sec)
> epoch 013: train 108.226 val 100.332 test 100.073 (209.1 sec)
> epoch 014: train 104.159 val 98.225 test 96.702 (209.3 sec)
> epoch 015: train 101.68

> epoch 133: train 28.542 val 41.007 test 41.434 (205.3 sec)
> epoch 134: train 28.439 val 40.133 test 40.614 (206.4 sec)
> epoch 135: train 28.052 val 41.220 test 42.058 (206.4 sec)
> epoch 136: train 28.013 val 40.254 test 40.628 (206.5 sec)
> epoch 137: train 27.546 val 40.672 test 41.168 (206.8 sec)
> epoch 138: train 27.234 val 39.695 test 39.773 (206.6 sec)
> epoch 139: train 27.143 val 39.068 test 39.803 (206.8 sec)
> epoch 140: train 26.820 val 39.781 test 40.294 (206.3 sec)
> epoch 141: train 26.661 val 40.505 test 40.806 (207.6 sec)
> epoch 142: train 26.438 val 39.987 test 40.541 (206.6 sec)
> epoch 143: train 26.241 val 39.320 test 40.172 (206.7 sec)
> epoch 144: train 25.896 val 38.840 test 39.355 (206.4 sec)
> epoch 145: train 25.812 val 39.397 test 40.125 (206.7 sec)
> epoch 146: train 25.389 val 39.450 test 39.938 (207.4 sec)
> epoch 147: train 25.232 val 38.391 test 38.997 (207.2 sec)
> epoch 148: train 25.061 val 38.299 test 38.898 (207.1 sec)
> epoch 149: train 24.90